In [3]:
import pandas as pd
import glob
import re

In [4]:
ls data/

equipment_A.csv  equipment_D.csv       sample_submit.csv  track_C.csv
equipment_B.csv  index_master.csv      track_A.csv        track_D.csv
equipment_C.csv  report_template.pptx  track_B.csv


In [5]:
path_list = glob.glob('./data/*.csv')

In [6]:
for path in path_list:
    print(re.match(r'.+/(.+).csv',path).group(1))

track_C
equipment_D
sample_submit
equipment_A
track_B
equipment_C
index_master
track_A
equipment_B
track_D


In [7]:
# データの読み込み
data_list = []
for path in path_list:
    data = pd.read_csv(path)
    data_list.append(data)

In [8]:
# データ作成
track_c = data_list[0]
equipment_d = data_list[1]
sample_submit = data_list[2]
equipment_a = data_list[3]
track_b = data_list[4]
equipment_c = data_list[5]
index_master = data_list[6]
track_a = data_list[7]
equipment_b = data_list[8]
track_d = data_list[9]

In [12]:
index_master.head()

,id,路線,date,キロ程
0,0,A,2018-04-01,10000
1,1,A,2018-04-01,10001
2,2,A,2018-04-01,10002
3,3,A,2018-04-01,10003
4,4,A,2018-04-01,10004


In [85]:
track_a.drop(['date', 'キロ程'], axis=1).corr()

,高低左,高低右,通り左,通り右,水準,軌間,速度
高低左,1.000000,0.859896,-0.024764,0.000893,0.127706,-0.008493,-0.001637
高低右,0.859896,1.000000,-0.013748,0.007806,-0.230668,-0.007818,-0.002890
通り左,-0.024764,-0.013748,1.000000,0.672010,0.110360,0.239638,-0.005872
通り右,0.000893,0.007806,0.672010,1.000000,0.114152,-0.186453,-0.003561
水準,0.127706,-0.230668,0.110360,0.114152,1.000000,0.004297,0.004581
軌間,-0.008493,-0.007818,0.239638,-0.186453,0.004297,1.000000,0.000767
速度,-0.001637,-0.002890,-0.005872,-0.003561,0.004581,0.000767,1.000000


In [121]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
model = LinearRegression()
sample_a = track_a.dropna()
X = sample_a['高低右'].as_matrix().reshape(-1, 1)
y = sample_a['高低左'].as_matrix().reshape(-1, 1)

In [122]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [123]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [125]:
from sklearn.metrics import mean_absolute_error

In [126]:
help(mean_absolute_error)

Help on function mean_absolute_error in module sklearn.metrics.regression:

mean_absolute_error(y_true, y_pred, sample_weight=None, multioutput='uniform_average')
    Mean absolute error regression loss
    
    Read more in the :ref:`User Guide <mean_absolute_error>`.
    
    Parameters
    ----------
    y_true : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    
    y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Estimated target values.
    
    sample_weight : array-like of shape = (n_samples), optional
        Sample weights.
    
    multioutput : string in ['raw_values', 'uniform_average']
        or array-like of shape (n_outputs)
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.
    
        'raw_values' :
            Returns a full set of errors in case of multioutput input.
    
        'uniform_average' :
          

In [128]:
predict = model.predict(x_test)
mean_absolute_error(y_test, predict)

0.6365623061793099

In [138]:
print(track_a.columns)
print(track_b.columns)
print(track_c.columns)
print(track_d.columns)

Index(['date', 'キロ程', '高低左', '高低右', '通り左', '通り右', '水準', '軌間', '速度'], dtype='object')
Index(['date', 'キロ程', '高低左', '高低右', '通り左', '通り右', '水準', '軌間', '速度'], dtype='object')
Index(['date', 'キロ程', '高低左', '高低右', '通り左', '通り右', '水準', '軌間', '速度'], dtype='object')
Index(['date', 'キロ程', '高低左', '高低右', '通り左', '通り右', '水準', '軌間', '速度'], dtype='object')


In [9]:
track_a['路線'] = 'A'
track_b['路線'] = 'B'
track_c['路線'] = 'C'
track_d['路線'] = 'D'

In [10]:
merge_date = pd.concat([track_a, track_b, track_c, track_d])

In [11]:
merge_data_gropuby = merge_date.groupby(['キロ程', '路線'])['高低左'].mean().reset_index()

In [2]:
submit_data = index_master.merge(merge_date, on=['キロ程', '路線'])

Interactive namespace is empty.


In [26]:
submit_data = index_master.merge(merge_data_gropuby, on=['路線', 'キロ程']).sort_values(by='id')

In [27]:
sample_submit

,0,-1.33
0,1,-1.14
1,2,-0.73
2,3,-0.04
3,4,0.84
4,5,1.78
5,6,2.39
6,7,2.24
7,8,1.16
8,9,-0.91
9,10,-3.26


In [31]:
submit = submit_data[['id', '高低左']]

In [37]:
submit.to_csv('./submit_test.csv', header=None, index=False)

In [38]:
pd.read_csv('./submit_test.csv').head()

,0,-1.3305747126436782
0,1,-1.136923
1,2,-0.727336
2,3,-0.042664
3,4,0.844023
4,5,1.779080
